<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN02/2.2-Klassifizierung_Movie-Reviews_PyTorch.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt


In [ ]:
import sys
import sys
import torch
import pickle
import pickle
import matplotlib
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from torchsummary import summary

from torch.utils.data import DataLoader, TensorDataset, random_split


In [ ]:
# Print versions in a compact form
print(f"Python version: {sys.version}")
print(f"torch: {torch.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")


# Filmkritiken klassifizieren: ein binäres Klassifizierungsbeispiel

Dieses Notebook enthält die Codebeispiele aus Kapitel 3, Abschnitt 5 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).

Die Zweiklassenklassifikation oder binäre Klassifikation ist möglicherweise die am weitesten verbreitete Art des maschinellen Lernens. In diesem Beispiel werden wir lernen, basierend auf dem Textinhalt der Rezensionen Filmkritiken in "positive" und "negative" Rezensionen zu klassifizieren.

## Der IMDB-Datensatz


Wir werden mit dem "IMDB-Datensatz" arbeiten, einem Satz von 50.000 hochpolarisierenden Rezensionen aus der Internet Movie Database. Sie sind aufgeteilt in 25.000 Rezensionen für das Training und 25.000 Rezensionen für das Testen, wobei jedes Set aus 50% negativen und 50% positiven Rezensionen besteht.

Warum haben wir diese zwei getrennten Trainings- und Test-Sets? 
- Sie sollten ein Modell für maschinelles Lernen **niemals mit denselben Daten testen, mit denen Sie es trainiert haben!** 
- Nur weil ein Modell bei seinen Trainingsdaten gut abschneidet, bedeutet das nicht, dass es auch bei Daten gut abschneidet, die es noch nie gesehen hat.
- Was und was ins wirklich interessiert, ist die Leistung unseres Modells bei neuen Daten (da Sie die Labels Ihrer Trainingsdaten bereits kennen - offensichtlich brauchen Sie Ihr Modell nicht, um diese vorherzusagen). 
- Es ist z. B. möglich, dass Ihr Modell am Ende nur ein Mapping zwischen Ihren Trainingsproben und deren Zielen speichert - was für die Aufgabe der Vorhersage von Zielen für nie zuvor gesehene Daten völlig nutzlos wäre. 




Genau wie der MNIST-Datensatz wird auch der IMDB-Datensatz mit `tensorflow.keras` ausgeliefert. Er wurde bereits vorverarbeitet: Die Bewertungen (Wortfolgen) wurden in Folgen von Ganzzahlen umgewandelt, wobei jede Ganzzahl für ein bestimmtes Wort in einem Wörterbuch steht.

Der folgende Code lädt den Datensatz (wenn Sie ihn zum ersten Mal ausführen, werden etwa 80 MB an Daten auf Ihren Rechner heruntergeladen):

In [ ]:
# Dateien laden aus Github
!wget -O imdb_dataset.pkl "https://github.com/ChristophWuersch/AppliedNeuralNetworks/raw/refs/heads/main/ANN02/Daten/imdb_dataset.pkl"
!wget -O imdb_word_index.pkl "https://github.com/ChristophWuersch/AppliedNeuralNetworks/raw/refs/heads/main/ANN02/Daten/imdb_word_index.pkl"

In [ ]:
# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the IMDB dataset

# Load the dataset from the pickle file
with open("imdb_dataset.pkl", "rb") as dataset_file:
    imdb_data = pickle.load(dataset_file)

# Load the word index from the pickle file
with open("imdb_word_index.pkl", "rb") as word_index_file:
    word_index = pickle.load(word_index_file)

# Access the dataset
train_data = imdb_data["train_data"]
train_labels = imdb_data["train_labels"]
test_data = imdb_data["test_data"]
test_labels = imdb_data["test_labels"]

print(f"Loaded {len(train_data)} training samples and {len(test_data)} test samples.")
print(f"Loaded word index with {len(word_index)} words.")

print("IMDB dataset and word index saved to pickle files.")


In [ ]:
# (b) We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# (c) We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])


Das Argument `num_words=10000` bedeutet, dass wir nur die 10.000 am häufigsten vorkommenden Wörter in den Trainingsdaten behalten werden. Seltene Wörter 
werden verworfen. Dies erlaubt uns, mit Vektordaten von überschaubarer Größe zu arbeiten.

Die Variablen `train_data` und `test_data` sind Listen von Bewertungen, wobei jede Bewertung eine Liste von Wortindizes ist (die eine Folge von Wörtern kodieren). 
Die Variablen `train_labels` und `test_labels` sind Listen von 0en und 1en, wobei 0 für "negativ" und 1 für "positiv" steht:

In [ ]:
train_data[1][:20]


In [ ]:
train_labels[0]


In [ ]:
train_data.shape


Da wir uns auf die 10.000 häufigsten Wörter beschränkt haben, wird kein Wortindex 10.000 überschreiten:

In [ ]:
max([max(sequence) for sequence in train_data])


Zum Spass können wir eine dieser Rezensionen schnell in englische Wörter zurückdekodieren:

In [ ]:
# (b) We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# (c) We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])


- (a) `word_index`  ist ein Dictionary, das den Wörtern einen Integerindex zuordnet.
- (b) Kehrt die Zuordnung um und ordnet Wortindizes Wörter zu.
- (c) Decodiert die Bewertung. Beachten Sie, dass die Indizes um drei Stellen verschoben sind, weil die Indizes 0, 1 und 2 für die Markierungen "padding", "start of sequence" und "unknown" reserviert sind.

In [ ]:
decoded_review


## Vorbereiten der Daten


Wir können keine Listen mit ganzen Zahlen in ein neuronales Netzwerk einspeisen. Wir müssen unsere Listen in Tensoren umwandeln. Es gibt zwei Möglichkeiten, dies zu tun:

* Wir könnten unsere Listen so auffüllen, dass sie alle die gleiche Länge haben, und sie in einen Integer-Tensor der Form `(samples, word_indices)` umwandeln, und dann als erste Schicht in unserem Netzwerk eine Schicht verwenden, die mit solchen ganzzahligen Tensoren umgehen kann (die Schicht `Embedding`, werden wir später ausführlich behandeln werden). 

* Wir könnten unsere Listen in einem Schritt kodieren, um sie in Vektoren von 0en und 1en zu verwandeln. Konkret würde das zum Beispiel bedeuten, dass wir die Sequenz `[3, 5]` in einen 10.000-dimensionalen Vektor zu verwandeln, der bis auf die Indizes 3 und 5, die Einsen wären, aus lauter Nullen bestehen würde. Dann könnten wir als Schicht in unserem Netzwerk eine `Dense`-Schicht verwenden, die in der Lage ist, Fliesskomma-Vektordaten zu verarbeiten.

Wir werden uns für die letztere Lösung entscheiden. Lassen Sie uns unsere Daten vektorisieren, was wir aus Gründen der Übersichtlichkeit manuell tun werden:

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0  # set specific indices of results[i] to 1s
    return results


# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)


So sehen unsere Stichproben jetzt aus:

In [ ]:
x_train[1]


Wir sollten auch unsere Beschriftungen vektorisieren, was ganz einfach ist:

In [ ]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")
y_test


In [ ]:
# Convert labels to tensors
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Create PyTorch datasets
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Combine into datasets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


Jetzt sind unsere Daten bereit, in ein neuronales Netzwerk eingespeist zu werden.

## Aufbau unseres Netzwerks


Unsere Eingabedaten sind einfach Vektoren, und unsere Beschriftungen sind Skalare (1en und 0en): Das ist die einfachste Konfiguration, die Sie je finden werden. Ein Typ von 
Netzes, das bei einem solchen Problem gut funktioniert, wäre ein einfacher Stapel voll verbundener (`Dense`) Schichten mit `relu`-Aktivierungen: `Dense(16, activation='relu')`

Das Argument, das an jede `Dense`-Schicht übergeben wird (16), ist die Anzahl der "versteckten Einheiten" der Schicht. Was ist eine versteckte Einheit? Es ist eine Dimension 
im Darstellungsraum der Schicht. Sie erinnern sich vielleicht aus dem vorigen Kapitel, dass jede solche `Dense`-Schicht mit einer `relu`-Aktivierung folgende Kette von Tensoroperationen implementiert 
die folgende Kette von Tensoroperationen implementiert:

`Output = relu(dot(W, input) + b)`


- 16 versteckte Einheiten (hiddne units) zu haben bedeutet, dass die Gewichtsmatrix `W` die Form `(input_dimension, 16)` haben wird, d.h. das Punktprodukt mit `W` projiziert die Eingabedaten auf einen 16-dimensionalen Darstellungsraum projizieren (und dann würden wir den Bias-Vektor `b` hinzufügen und die Operation `relu` anwenden). 
- Sie können Sie können die Dimensionalität Ihres Repräsentationsraums intuitiv als "wie viel Freiheit Sie dem Netzwerk beim Lernen interner Repräsentationen einräumen" verstehen. interne Repräsentationen zu lernen". Wenn Sie mehr versteckte Einheiten (einen höherdimensionalen Repräsentationsraum) haben, kann Ihr Netz komplexere Repräsentationen lernen, aber es macht Ihr Netz rechenintensiver und kann zum Erlernen unerwünschter Muster führen (Muster, die die Leistung bei den Trainingsdaten verbessern, aber nicht bei den Testdaten).



Es gibt zwei wichtige Architektur-Entscheidungen, die für einen solchen Stapel von dichten Schichten getroffen werden müssen:

* Wie viele Schichten verwendet werden sollen.
* Wie viele "versteckte Einheiten" für jede Schicht gewählt werden sollen.

Bezüglich der Architektur eines solchen Stapels von Dense-Layern müssen zwei wichtige **Designentscheidungen** getroffen werden:
- wie viele Layer verwendet werden und
- wie viele verdeckte Einheiten jeder Layer besitzt.



In den folgenden Lektionen werden  Sie die formalen Regeln kennenlernen, nach denen Sie sich bei diesen Entscheidungen richten können. Fürs Erste müssen Sie darauf vertrauen, dass die folgende Wahl die richtige ist:
- zwei zwischenliegende Layer mit jeweils 16 verdeckten Einheiten
- ein dritter Layer zur Ausgabe der skalaren Vorhersage der aktuellen Filmbewertung

Die zwischenliegenden Layer verwenden als Aktivierungsfunktion `relu`, und der letzte Layer nutzt zur Aktivierung eine **Sigmoidfunktion**, um eine Wahrscheinlichkeit ausgeben zu können (einen Wert zwischen 0 und 1, der angibt, wie wahrscheinlich
es ist, dass das fragliche Sample den Zielwert 1 besitzt, also wie hoch die Wahrscheinlichkeit ist, dass die Bewertung positiv ist). Die relu-Funktion (Rectified Linear Units, rektifizierte Lineareinheiten) sorgt dafür, dass negative Werte auf null gesetzt werden (siehe Abbildung 3.4). Die Sigmoidfunktion »quetscht« beliebige Werte in das Intervall `[0, 1]` (siehe Abbildung), damit die Ausgabe als Wahrscheinlichkeit interpretiert werden kann.

## Was sind Aktivierungsfunktionen, und wofür werden sie benötigt?

Ohne eine Aktivierungsfunktion wie relu (die auch als Nichtlinearität bezeichnet wird) würde der Dense-Layer aus zwei linearen Operationen bestehen – dem Tensorprodukt und einer Addition:

`output = dot(W, input) + b`

- Der Layer könnte in diesem Fall nur lineare Transformationen (affine Abbildungen) der Daten erlernen: Der *Hypothesenraum* des Layers wäre die *Menge aller möglichen linearen Transformationen* der Eingabedaten in einen 16-dimensionalen Raum. Ein solcher Hypothesenraum ist zu beschränkt und würde nicht von mehreren Repräsentations-Layern profitieren, denn auch ein großer Stapel linearer Layer stellt noch immer eine lineare Operation dar: Das Hinzufügen weiterer Layer würde den Hypothesenraum nicht vergrössern.

- Um Zugang zu einem sehr viel umfassenderen Hypothesenraum zu erlangen, der von vielschichtigen Repräsentationen profitieren kann, ist eine **Nichtlinearität bzw. eine Aktivierungsfunktion** erforderlich. Beim Deep Learning ist `relu` die
verbreitetste Aktivierungsfunktion, es gibt jedoch noch viele weitere, die ähnlich seltsame Bezeichnungen besitzen, wie `prelu`, `elu` usw.

Here's what our network looks like:

![3-layer network](https://s3.amazonaws.com/book.keras.io/img/ch3/3_layer_network.png)

Und hier ist die Keras-Implementierung, dem MNIST-Beispiel, das Sie zuvor gesehen haben, sehr ähnlich:

In [ ]:
# Define the model
class SentimentModel(nn.Module):
    def __init__(self):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(10000, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)


# Instantiate the model and move it to the device
model = SentimentModel().to(device)
print(model)


In [ ]:
# Summarize the model
summary(
    model, input_size=(1, 10000)
)  # Adjust input_size as per your model's requirement

# Visualize the model architecture
x = torch.randn(1, 10000)  # Adjust input size as per your model's requirement
y = model(x)


# Summarize the model
summary(
    model, input_size=(1, 10000)
)  # Adjust input_size as per your model's requirement


Abschliessend müssen Sie noch eine **Verlustfunktion** und einen **Optimierer** auswählen.

- Da wir es hier mit einer Binärklassifizierung zu tun haben und die Ausgabe des NNs eine Wahrscheinlichkeit ist (der letzte Layer des NNs besitzt nur eine verdeckte Einheit und wird durch eine Sigmoidfunktion aktiviert), ist es am besten, die **binäre Kreuzentropie** (`binary_crossentropy`) als Verlustfunktion zu verwenden. Dabei handelt es sich jedoch nicht um die einzige brauchbare Lösung.
- Sie könnten beispielsweise auch den mittleren quadratischen Fehler (`mean_squared_error`) benutzen. Für Modelle, die Wahrscheinlichkeiten ausgeben, ist die Kreuzentropie jedoch für gewöhnlich die beste Wahl. Die Kreuzentropie ist eine Größe, die einem Teilgebiet der Informationstheorie entstammt, das die Differenz zwischen Wahrscheinlichkeitsverteilungen bemisst oder, wie im vorliegenden Fall, zwischen der beobachteten Wahrscheinlichkeitsverteilung und der Vorhersage.


## Kreuzentropie $H(p,q)$

Die Kreuzentropie der Verteilung $q$ relativ zu einer Verteilung $p$ über eine gegebene Menge ist wie folgt definiert:

$$ H(p,q)=-\mathbb{E}_{p}[\log q]$$

Bei **Klassifizierungsproblemen** wollen wir die Wahrscheinlichkeit verschiedener Ergebnisse schätzen. Wenn die geschätzte Wahrscheinlichkeit $q_i(x)$ des Ergebnisses $i$ ist, während die Häufigkeit (empirische Wahrscheinlichkeit) des Ergebnisses $i$ in der Trainingsmenge $p_{i}$ ist, und es $N$ bedingt unabhängige Stichproben in der Trainingsmenge gibt, dann ist die Wahrscheinlichkeit des Modells auf der Trainingsmenge:

$$\prod_i q_i(x)^{N p_i}$$



so wird die **log-likelihood** (dividiert durch $N$) gerade die Kreuzentropie,

$$ \frac{1}{N}\log \prod_{i}q_{i}^{Np_{i}}=\sum_{i}p_{i}\log q_{i}(x)=-H(p,q)$$

sodass die Maximierung der Wahrscheinlichkeit (likelihood) dasselbe ist wie die Minimierung der Kreuzentropie.

$$
\begin{aligned}
J(\mathbf{w}) &=\frac{1}{N} \sum_{n=1}^{N} H(p_{n},q_{n}) 
= -\frac{1}{N} \sum_{n=1}^{N} \bigg\lbrace y_{n} \log \left[\hat{y}_{n}(x)\right]+(1-y_{n}) \log \left[ 1-\hat{y}_{n}(x)\right] \bigg\rbrace\,
\end{aligned}
$$
                                                                                                                                              

In [ ]:
# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Metric tracking
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []


Wir übergeben unseren Optimierer, die Verlustfunktion und die Metriken als Strings, was möglich ist, weil `rmsprop`, `binary_crossentropy` und `accuracy` als Teil von Keras mitgeliefert werden. 

Gelegentlich möchte man die Parameter des Optimierers konfigurieren oder eine benutzerdefinierte Verlustfunktion
bzw. Kennzahl übergeben. Ersteres kann man durch Übergabe einer Instanz einer Optimierer-Klasse als optimizer-Argument erreichen.

## Validierung des Ansatzes

Um während des Trainings die Korrektklassifizierungsrate des Modells für unbekannte Daten zu überwachen, halten wir 10.000 Samples der ursprünglichen Trainingsdaten zurück, die dann als Validierungsmenge verwendet wird.

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]


Nun wird das Modell 20 Epochen lang mit kleinen Stapeln trainiert, die jeweils 512 Samples enthalten (20 Durchläufe sämtlicher Samples in den Tensoren `x_train` und `y_train`). Gleichzeitig überwachen wir den Wert der Verlustfunktion und die Korrektklassifizierungsrate für die zurückgehaltenen 10.000 Samples. Zu diesem Zweck übergeben wir als validation_data-Argument die Validierungsdatenmenge.

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).float()  # Convert probabilities to binary predictions
    return (y_pred == y_true).float().mean()


# Training loop
epochs = 20

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Forward pass
        y_pred = model(x_batch).squeeze()
        loss = criterion(y_pred, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update metrics
        train_loss += loss.item() * x_batch.size(0)
        train_acc += calculate_accuracy(y_pred, y_batch).item() * x_batch.size(0)

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    # Validation step
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_pred = model(x_batch).squeeze()
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item() * x_batch.size(0)
            val_acc += calculate_accuracy(y_pred, y_batch).item() * x_batch.size(0)

    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)

    # Print epoch results
    print(
        f"Epoch {epoch + 1}/{epochs} - "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )


Auf einer CPU dauert die Berechnung weniger als 2 Sekunden pro Epoche – das Training ist nach 20 Sekunden erledigt. Am Ende jeder Epoche gibt es eine kurze Verzögerung, weil das Modell den Wert der Verlustfunktion und die Korrektklassifizierungsrate
für die 10.000 Samples der Validierungsdatenmenge berechnet.


Das Dictionary enthält vier Einträge: einen für jede Kennzahl, die während des Trainings und der Validierung überwacht wurde. Die beiden folgenden Listings zeigen, wie man mit `Matplotlib` den Wert der Verlustfunktion bzw. der
Korrektklassifizierungsrate beim Training und bei der Validierung ausgeben und vergleichen kann. Ihre Ergebnisse können aufgrund einer anderen zufälligen Initialisierung des NNs ein wenig davon abweichen.

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(range(1, epochs + 1), train_loss_history, label="Training Loss", marker="o")
plt.plot(range(1, epochs + 1), val_loss_history, label="Validation Loss", marker="o")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(range(1, epochs + 1), train_acc_history, label="Training Accuracy", marker="o")
plt.plot(range(1, epochs + 1), val_acc_history, label="Validation Accuracy", marker="o")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()


Das Modell wird auf dem Testdatensatz bewertet, um die endgültige Leistung zu messen. Der Testverlust und die Testgenauigkeit werden berechnet.

In [ ]:
# Evaluate on test set
model.eval()
test_loss = 0.0
test_acc = 0.0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_pred = model(x_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        test_loss += loss.item() * x_batch.size(0)
        test_acc += calculate_accuracy(y_pred, y_batch).item() * x_batch.size(0)

test_loss /= len(test_loader.dataset)
test_acc /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Das trainierte Modell wird als Datei gespeichert, um es später laden und wiederverwenden zu können.

In [ ]:
# Save the model
model_path = "sentiment_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


### Bias-Variance Tradeoff
Wie Sie sehen, sinkt der Wert der Verlustfunktion beim Training mit jeder Epoche, während die Korrektklassifizierungsrate zunimmt. Das ist bei einer Optimierung durch ein Gradientenabstiegsverfahren auch zu erwarten – die Grösse, die man zu
minimieren versucht, sollte bei jedem Durchlauf kleiner werden. 

- Der Wert der Verlustfunktion und die Korrektklassifizierungsrate bei der Validierung zeigen dieses Verhalten allerdings nicht: Sie erreichen offenbar in der vierten Epoche die besten Werte. 
- Hierbei handelt es sich um ein Beispiel für das, vor dem ich Sie bereits gewarnt habe: Ein Modell, das mit den Trainingsdaten eine bessere Leistung erzielt, funktioniert nicht unbedingt auch mit unbekannten Daten besser.
Genauer gesagt, liegt hier eine **Überanpassung (overfitting, high variance)** vor: Nach der zweiten Epoche wird
das Modell zu stark an die Trainingsdaten angepasst und erlernt Repräsentationen, die den Trainingsdaten zu eigen sind, sich aber nicht auf unbekannte Daten verallgemeinern lassen.

**Regularisierung durch "early stopping"**: In diesem Fall könnten Sie das Training nach drei Epochen abbrechen, um eine Überanpassung zu verhindern. Sie können im Allgemeinen eine Reihe verschiedener Verfahren zum Abschwächen der Überanpassung einsetzen, auf die wir in
Kapitel 4 näher eingehen.

In [ ]:
# Early stopping class
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


In [ ]:
# Training routine with early stopping
def train_with_early_stopping(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    patience=5,
    max_epochs=50,
    device="cpu",
):
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    train_losses = []
    val_losses = []

    for epoch in range(max_epochs):
        model.train()
        train_loss = 0.0

        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch).squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x_batch.size(0)

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch).squeeze()
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * x_batch.size(0)

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        print(
            f"Epoch {epoch + 1}/{max_epochs} - Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}"
        )

        # Check early stopping
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break

    return train_losses, val_losses


In [ ]:
model = SentimentModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

train_losses, val_losses = train_with_early_stopping(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    patience=5,
    max_epochs=50,
    device=device,
)


Dieser ziemlich naive Ansatz erreicht eine Korrektklassifizierungsrate von 88%.
Mit Ansätzen, die dem Stand der Technik entsprechen, lassen sich aber auch
Werte von bis zu 95% erzielen.



# Einführung in PyTorch Lightning

PyTorch Lightning ist ein High-Level-Framework, das auf PyTorch aufbaut. Es vereinfacht die Strukturierung von Deep-Learning-Projekten, indem es Boilerplate-Code für Training, Validierung und Logging eliminiert. Mit Lightning kann man sich auf das Wesentliche konzentrieren: die Modelllogik und die Forschung.

### **Vorteile von PyTorch Lightning**:
- **Modularer Aufbau:** Lightning-Module kapseln Modelle, Optimierer und Trainingslogik.
- **Automatisches Management:** Training, Validierung und Testläufe werden automatisch durch den `Trainer` verwaltet.
- **Callbacks:** Unterstützt nützliche Callbacks wie Early Stopping und Checkpointing.
- **Hardware-Unterstützung:** Läuft problemlos auf CPUs, GPUs und TPUs mit minimalem Aufwand.



###  **Überblick über den Code**

### Modelldefinition
Das Modell wird in einer Lightning-Klasse definiert, die von `pl.LightningModule` erbt. Diese Klasse kapselt:
- **Forward-Methode:** Definiert, wie die Eingaben durch das Modell fließen.
- **Training Step:** Berechnet den Verlust während des Trainings.
- **Validation Step:** Berechnet den Verlust für die Validierungsdaten.
- **Optimizers:** Gibt den Optimierer zurück, der während des Trainings verwendet wird.



In [ ]:
# Example usage with PyTorch Lightning Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class SentimentModel(pl.LightningModule):
    def __init__(self):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(10000, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
        self.criterion = nn.BCELoss()

        # Initialize metric storage
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.sigmoid(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc, prog_bar=True)
        return {"loss": loss, "train_acc": acc}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.criterion(y_hat, y)
        acc = ((y_hat > 0.5).float() == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)
        return {"val_loss": loss, "val_acc": acc}

    def on_train_epoch_end(self):
        # Access logged training metrics
        self.train_losses.append(self.trainer.callback_metrics["train_loss"].item())
        self.train_accuracies.append(self.trainer.callback_metrics["train_acc"].item())

    def on_validation_epoch_end(self):
        # Access logged validation metrics
        self.val_losses.append(self.trainer.callback_metrics["val_loss"].item())
        self.val_accuracies.append(self.trainer.callback_metrics["val_acc"].item())

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.001)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512)

# Early stopping callback
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=True, mode="min")

# Initialize the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    callbacks=[early_stopping],
    log_every_n_steps=1,
)

# Instantiate the model
model = SentimentModel()

# Train the model
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


In [ ]:
model.train_losses


In [ ]:
model.val_losses


In [ ]:
epochs = np.arange(0, len(model.train_losses) + 1)
epochs


In [ ]:
# Plot metrics after training
def plot_metrics_from_model(model):
    epochs_train = np.arange(1, len(model.train_losses) + 1)
    epochs_val = np.arange(1, len(model.val_losses) + 1)

    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs_train, model.train_losses, label="Train Loss")
    plt.plot(epochs_val, model.val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss over Epochs")
    plt.legend()
    plt.grid()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs_train, model.train_accuracies, label="Train Accuracy")
    plt.plot(epochs_val, model.val_accuracies, label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy over Epochs")
    plt.legend()
    plt.grid()

    plt.tight_layout()
    plt.show()


In [ ]:
plot_metrics_from_model(model)


## Weitere Experimente (Aufgaben)

Die folgenden Experimente werden Sie davon überzeugen, dass die für die Architektur
getroffenen Designentscheidungen durchaus vernünftig waren, wenngleich
es noch Verbesserungsmöglichkeiten gibt:
- Sie haben zwei verdeckte Layer verwendet. Probieren Sie aus, nur einen oder drei zu benutzen, und überprüfen Sie, wie sich das auf die Korrektklassifizierungsrate der Validierungs- bzw. der Testdatenmenge auswirkt.
- Probieren Sie aus, weniger oder weitere verdeckte Einheiten zu verwenden: `8, 32, 64` usw.
- Probieren Sie aus, statt der binären Kreuzentropie (binary_crossentropy) den mittleren quadratischen Fehler (Mean Squared Error, mse) zu verwenden. 
- Probieren Sie aus, statt der `relu`-Funktion tanh (Tangens hyperbolicus) als Aktivierungsfunktion zu verwenden (diese Aktivierungsfunktion war in den Anfangstagen der NNs verbreitet).

## Zusammenfassung

Nehmen Sie Folgendes aus diesem Abschnitt mit:

1. Für gewöhnlich ist mit der Vorverarbeitung der Rohdaten durchaus etwas Aufwand verbunden, damit sie – als Tensoren – in ein NN eingespeist werden können. Sequenzen von Wörtern können als binäre Vektoren codiert werden, aber meistens gibt es noch weitere Möglichkeiten, sie zu codieren.
2. Mit Stapeln (batches) von `Dense`-Layern mit `relu`-Aktivierungsfunktion lässt sich eine Vielzahl von Aufgaben lösen (z.B. Stimmungsanalysen = sentiment analysis). Sie werden sie häufig nutzen.
3. Bei **binären Klassifizierungsaufgaben** (zwei Klassen als mögliche Ausgabe) sollte das NN mit einem `Dense`-Layer mit einer verdeckten Einheit und **sigmoid-Aktivierung** enden. Die Ausgabe sollte ein Skalar zwischen `0` und `1` sein, der eine Wahrscheinlichkeit angibt.
4. Bei einer solchen sigmoiden Ausgabe einer binären Klassifizierungsaufgabe sollten Sie als **Verlustfunktion** die **binäre Kreuzentropie** (`binary_crossentropy`) verwenden.
5. Der `rmsprop`-**Optimierer** ist im Allgemeinen, unabhängig von der gegebenen Aufgabe, eine gute Wahl. Eine Sache weniger, um die Sie sich kümmern müssen.
6. Wenn die Leistung eines NNs bei den Trainingsdaten zunimmt, kommt es irgendwann zu einer **Überanpassung**, die zu immer schlechteren Ergebnissen bei unbekannten Daten führt. Bei Daten, die nicht zur Trainingsmenge gehören, sollten Sie stets die Leistung überwachen.
